In [1]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import joblib
import requests
import pandas as pd
from datetime import datetime, timedelta
import json

In [2]:
# rf_model = joblib.load("../models/no2/random_forest_model.pkl")
xgb_model = joblib.load("../models/no2/xgboost_model.pkl")

In [3]:
x_scaler = joblib.load("../scalers/no2/x_scaler.pkl")
y_scaler = joblib.load("../scalers/no2/y_scaler.pkl")

In [4]:
latitude = 41.9981
longitude = 21.4254

url = f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&hourly=temperature_2m,relative_humidity_2m,windspeed_10m"
response = requests.get(url)

if response.status_code != 200:
    raise Exception("Error fetching weather data")

data = response.json()

hourly_data = {
    "time": data["hourly"]["time"],  
    "temperature": data["hourly"]["temperature_2m"],
    "humidity": data["hourly"]["relative_humidity_2m"],
    "wind_speed": data["hourly"]["windspeed_10m"]
}

df = pd.DataFrame(hourly_data)

df["time"] = pd.to_datetime(df["time"])

current_time = datetime.now()
next_2_days = current_time + timedelta(days=2)

df_next_2_days = df[(df["time"] >= current_time) & (df["time"] <= next_2_days)].copy()

df_next_2_days.loc[:, "hour"] = df_next_2_days["time"].dt.hour
df_next_2_days.loc[:, "year"] = df_next_2_days["time"].dt.year
df_next_2_days.loc[:, "month"] = df_next_2_days["time"].dt.month
df_next_2_days.loc[:, "day"] = df_next_2_days["time"].dt.day


X = df_next_2_days[['hour', 'year', 'month', 'day', 'temperature', 'humidity', 'wind_speed']]
X = x_scaler.transform(X)

# no2_rf_predictions = rf_model.predict(X)
no2_xgb_predictions = xgb_model.predict(X)


In [5]:
no2_pred = y_scaler.inverse_transform(no2_xgb_predictions.reshape(-1, 1))

In [7]:
no2_pred.shape

(48, 1)

In [6]:
print(json.dumps({"no2_predictions": no2_pred.tolist()}))

{"no2_predictions": [[22.582212448120117], [21.754093170166016], [18.005537033081055], [14.94070816040039], [14.450027465820312], [14.253520965576172], [15.050960540771484], [17.429523468017578], [17.37937355041504], [19.133617401123047], [18.939517974853516], [19.85582733154297], [22.923751831054688], [23.863712310791016], [25.369091033935547], [34.42451858520508], [34.50710678100586], [38.5930061340332], [40.02341842651367], [40.00715255737305], [37.61058044433594], [34.71978759765625], [30.902910232543945], [30.09906005859375], [30.06190299987793], [27.492822647094727], [24.937166213989258], [20.786422729492188], [20.21465301513672], [20.205121994018555], [20.13465118408203], [23.572956085205078], [26.003786087036133], [29.494369506835938], [30.870588302612305], [31.005640029907227], [30.484912872314453], [30.361190795898438], [32.370513916015625], [39.00113296508789], [43.882537841796875], [39.277442932128906], [41.2913932800293], [38.845699310302734], [38.91877365112305], [37.3972